In [ ]:
import csv
import requests
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from threading import Thread

In [ ]:
# driver = webdriver.Edge()
driver = webdriver.Chrome(r'your_driver_path\chromedriver')
driver.get('https://www.amazon.eg/s?i=fashion&fs=true&language=en')

In [ ]:
# get the whole div that carrying all the details of the product
wholeDiv = driver.find_element(By.XPATH,'//div[@class="s-card-container s-overflow-hidden aok-relative puis-expand-height puis-include-content-margin puis s-latency-cf-section s-card-border"]')

In [ ]:
# print properities of a single product
print(wholeDiv.get_property('innerHTML'))

In [16]:
import time
def offload_images(i,j):
    try:
        img_data = requests.get(i).content
        with open(f'images/{j}.jpg', 'wb') as handler:
            handler.write(img_data)
    except:
        pass
def element_finder(i,j):
    global results
    rating = None
    txt = i.get_property('innerHTML')
    y = re.search('"([0-9.]+) out of 5 stars"',txt)
    if y:
        rating = float(y.groups()[0])
        
    try:
        price = float(i.find_element(By.CLASS_NAME, "a-price-whole").get_attribute('innerText')[:-2])
    except:
        price = None
    name = i.find_element(By.CLASS_NAME, "a-size-base-plus").get_attribute('innerText')
    # image
    
    img = i.find_element(By.CLASS_NAME, "s-image")
    # download images into /images folder
    try:
        Thread(target=offload_images, args=(img.get_attribute('src'),j)).start()
    except:
        pass
    # write data into csv file
    results[j] =  [name,price,rating,f'images/{j}.jpg','Men','Accessories',name]
with open('.\File.csv', 'w',encoding='UTF-8',newline='') as csvfile:
    csvwriter = csv.writer(csvfile, delimiter=',')
    # header of the file
    csvwriter.writerow(['Product Name','Price','Rating','Img Path','Category','Category Name','Description'])
    j = 0
    k = 1
    
    while True:
        wholeDiv = driver.find_elements(By.XPATH,'//div[@class="s-card-container s-overflow-hidden aok-relative puis-expand-height puis-include-content-margin puis s-latency-cf-section s-card-border"]')
        results : dict[int, list]= {}
        threads : list[Thread] = []
        for i in wholeDiv:
            # use threading with element_finder function to speed up the process
            threads+=[Thread(target=element_finder, args=(i,j))]
            j += 1
        [thread.start() for thread in threads]
        [thread.join() for thread in threads]
        for x in results.keys():
            csvwriter.writerow(results[x])
        next = driver.find_element(By.CLASS_NAME, "s-pagination-next")
        if next.get_attribute('aria-disabled'):
            print("Ending!")
            break
        next.click()
        print(f"Page {k} done")
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, 's-pagination-next')))
        k+=1